# Image Classifier - Deep Learning

## Table of Contents

## 0. Problem Statement <a id='c0'></a>

The objective of this exercise is to successfully classify an animal into dog or cat with neural network model.

## 1. Importing libraries <a id='c1'></a>

In [3]:
import requests
import zipfile
import io
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt
import shutil

## 2. Data Collection <a id='c2'></a>

In [ ]:
URL = "https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip"
response = requests.get(URL)
response.raise_for_status()
output_dir = 'dogs-vs-cats'

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(output_dir)
train_dir = os.path.join(output_dir, output_dir,'train')

df = 